In [0]:
%pip install -qU langchain langchain-community langchain-core sqlalchemy langchain_experimental matplotlib mlflow langgraph fsspec huggingface_hub youtube-search openai langchain-databricks databricks-sdk langchain_openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython() 

In [0]:
import mlflow
import langchain
from mlflow.models import infer_signature
from mlflow import MlflowClient
from mlflow.models import validate_serving_input

In [0]:
%run ./graph

In [0]:
# load graph 
new_app = load_graph()

In [0]:
######## model signature ######## 

message = "get a youtube video on customer service greeting"
input = {"messages": [{"role": "user", "content": message}]}
output = new_app.invoke(input)
signature = infer_signature(input, output)

print(signature)

inputs: 
  ['messages': Array({content: string (required), role: string (required)}) (required)]
outputs: 
  [string (required)]
params: 
  None



In [0]:
######## Log LangGraph using Model from Code ######## 


catalog_name = 'llmcup2024'
schema_name = "default"

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog_name}.{schema_name}.agentqaxv2"



with mlflow.start_run() as run:
    
    model_info = mlflow.langchain.log_model(
        lc_model="./graph", # Path to our model Python file
        artifact_path="langgraph",
        registered_model_name= model_name, # Add model name to register it
        signature=signature,

    )

    model_uri = model_info.model_uri

2024/10/24 07:54:01 WARNING mlflow.langchain.databricks_dependencies: Unable to detect Databricks dependencies. Set logging level to DEBUG to see the full traceback.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Successfully registered model 'llmcup2024.default.agentqaxv2'.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '1' of model 'llmcup2024.default.agentqaxv2'.
2024/10/24 07:54:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-finch-143 at: https://dbc-b3a32eda-16b0.cloud.databricks.com/ml/experiments/1422264621101654/runs/198b5c50c1a54946aae1b74d24e67a71.
2024/10/24 07:54:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-b3a32eda-16b0.cloud.databricks.com/ml/experiments/1422264621101654.


In [0]:
######## Check model version ######## 

# Set the registry URI to Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# Initialize the MLflow client
client = MlflowClient()

# Get the latest version number of the specified model
model_version_infos = client.search_model_versions("name = '%s'" % model_name)
if model_version_infos:
    latest_model_version = max([model_version_info.version for model_version_info in model_version_infos])
else:
    raise(BaseException("Error: Model not created, verify if 00-Build-Model script ran successfully!"))

# Print the latest model version
print(f"Model : {model_name}, Latest model version: {latest_model_version}")

Model : llmcup2024.default.agentqaxv2, Latest model version: 1


/home/spark-73bab0af-d052-4e47-a9f0-1b/.ipykernel/52959/command-2586615384031825-4153226305:9: LangChainDeprecationWarning: The class `ChatDatabricks` was deprecated in LangChain 0.3.3 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-databricks package and should be used instead. To use it run `pip install -U :class:`~langchain-databricks` and import as `from :class:`~langchain_databricks import ChatDatabricks``.
  chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-1-405b-instruct", max_tokens=4000)
/home/spark-73bab0af-d052-4e47-a9f0-1b/.ipykernel/52959/command-2586615384031825-4153226305:73: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=feedback_prompt, verbose=False)


In [0]:
# Validate the serving payload works on the model
validate_serving_input(model_uri, input)

[{'messages': [{'role': 'user',
    'content': 'get a youtube video on customer service greeting'},
   {'content': '',
    'additional_kwargs': {'tool_calls': [{'id': 'call_63f582b5-c0bb-436a-98c7-9c8419dbbf36',
       'type': 'function',
       'function': {'name': 'youtube_search',
        'arguments': '{"query": "customer service greeting"}'}}]},
    'response_metadata': {'prompt_tokens': 775,
     'completion_tokens': 17,
     'total_tokens': 792},
    'type': 'ai',
    'name': None,
    'id': 'run-88830ba3-6038-4866-85a0-2f84107f8480-0',
    'example': False,
    'tool_calls': [{'name': 'youtube_search',
      'args': {'query': 'customer service greeting'},
      'id': 'call_63f582b5-c0bb-436a-98c7-9c8419dbbf36',
      'type': 'tool_call'}],
    'invalid_tool_calls': [],
    'usage_metadata': None},
   {'content': "['https://www.youtube.com/watch?v=oe6OXbzPj-I&pp=ygUZY3VzdG9tZXIgc2VydmljZSBncmVldGluZw%3D%3D', 'https://www.youtube.com/watch?v=LRJXMKZ4wOw&pp=ygUZY3VzdG9tZXIgc2Vydmlj

In [0]:
import mlflow
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

mlflow.set_experiment(experiment_id="1422264621101654")
mlflow.langchain.autolog()

In [0]:
# Load the model using the specified model URI
model_uri = 'runs:/198b5c50c1a54946aae1b74d24e67a71/langgraph'

loaded_model = mlflow.langchain.load_model(model_uri)

# Define the input message for the model
message = "get a youtube video on customer service greeting"
input = {"messages": [{"role": "user", "content": message}]}

# Invoke the loaded model with the input message
response = loaded_model.invoke(input)

# Print the content of the last message in the response
display(response["messages"][-1].content)

2024-10-27 01:49:33,924 52959 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/client/core.py", line 1718, in config
    resp = self._stub.Config(req, metadata=self.metadata())
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 439, in result
    raise self
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 1193, in with_call
    return _end_unary_response_blocking(state, call, True, None)
 

'Here are the YouTube video results for "customer service greeting":\n\n1. **"Customer Service Greeting Examples"** by CareerBuilder (https://www.youtube.com/watch?v=oe6OXbzPj-I)\n2. **"Customer Service Greeting Script"** by ServiceSkills (https://www.youtube.com/watch?v=b6GISKDhTps)\n\nYou can click on the links to watch the videos and learn more about customer service greetings!'

Trace(request_id=tr-db23eb5b510e44f38e0bd6fc8f4307cf)

### Mlflow run model log
<img src ='./model_log.png'>

### Mlflow traces
<img src ='./traces.png'>

### Register model to catalog
<img src ='./register_model.png'>